In [3]:
# backend/main.py
import os
import time
import json
import math
from typing import List, Optional, Dict, Any
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pathlib import Path

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
import faiss
import requests

# ------------------ Config ------------------
CORPUS_CSV = os.environ.get("CORPUS_CSV")
DEFAULT_CANDIDATES = 5
TOPK = int(os.environ.get("TOPK", "5"))
EMB_MODEL_NAME = os.environ.get("EMB_MODEL", "sentence-transformers/all-MiniLM-L6-v2")
SD_API_URL = os.environ.get("SD_API_URL")  # e.g., "http://127.0.0.1:8000/generate" from Track-A

DATA_DIR = Path(os.environ.get("DATA_DIR", "."))

# ------------------ App & State ------------------
app = FastAPI(title="Week7 Track-B Backend")

class QARequest(BaseModel):
    query: str
    top_k: int = TOPK

class MultiHopRequest(BaseModel):
    query: str
    top_k: int = TOPK

class SDRequest(BaseModel):
    prompt: str
    negative_prompt: Optional[str] = "nsfw, watermark, logo"
    steps: int = 20
    guidance: float = 7.0
    height: int = 512
    width: int = 512
    seed: Optional[int] = 123

class RAGIndex:
    def __init__(self):
        self.model = SentenceTransformer(EMB_MODEL_NAME)
        self.df = None
        self.index = None
        self.text_col = None
        self.id_col = None

    def load_csv(self, path: str):
        df = pd.read_csv(path)
        # choose a text column
        for cand in ["text", "content", "chunk", "passage", "body"]:
            if cand in df.columns:
                self.text_col = cand
                break
        if self.text_col is None:
            # fallback: use the longest string-like column
            str_cols = [c for c in df.columns if df[c].dtype == object]
            if not str_cols:
                raise ValueError("No text-like column found in CSV.")
            self.text_col = max(str_cols, key=lambda c: df[c].astype(str).str.len().sum())

        # Optional id column
        for cand in ["id", "doc_id", "source", "filename"]:
            if cand in df.columns:
                self.id_col = cand
                break

        # drop NA
        df = df[df[self.text_col].notna()].reset_index(drop=True)
        self.df = df

        # embed + faiss
        embs = self.model.encode(df[self.text_col].astype(str).tolist(), batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
        dim = embs.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(embs.astype(np.float32))
        self.index = index

    def search(self, q: str, k: int = TOPK):
        q_emb = self.model.encode([q], convert_to_numpy=True, normalize_embeddings=True)
        D, I = self.index.search(q_emb.astype(np.float32), k)
        results = []
        for score, idx in zip(D[0], I[0]):
            row = self.df.iloc[int(idx)]
            results.append({
                "score": float(score),
                "text": str(row[self.text_col]),
                "id": str(row[self.id_col]) if self.id_col else str(int(idx))
            })
        return results

index = RAGIndex()
LOGS_PATH = Path("reports/metrics.json")
LOGS_PATH.parent.mkdir(parents=True, exist_ok=True)

def log_event(kind: str, payload: Dict[str, Any]):
    now = time.time()
    if LOGS_PATH.exists():
        data = json.loads(LOGS_PATH.read_text())
    else:
        data = {"events": []}
    data["events"].append({"ts": now, "type": kind, "payload": payload})
    LOGS_PATH.write_text(json.dumps(data, indent=2))

# Try to preload CSV
try_paths = [p for p in [CORPUS_CSV, "./corpus.csv", "/content/corpus.csv"] if p]
for p in try_paths:
    try:
        if Path(p).exists():
            index.load_csv(p)
            print(f"[RAG] Loaded corpus from: {p}")
            break
    except Exception as e:
        print("Corpus load error:", e)

@app.get("/health")
def health():
    return {"ok": True, "has_index": index.index is not None}

@app.post("/qa")
def qa(req: QARequest):
    if index.index is None:
        raise HTTPException(400, "Index not loaded. Upload corpus via app or set CORPUS_CSV.")
    t0 = time.perf_counter()
    hits = index.search(req.query, k=req.top_k)
    # quick synthesis (extractive-ish)
    snippet = " ".join(h["text"] for h in hits[:3])[:1200]
    answer = f"Top evidence:\n" + "\n\n---\n\n".join([f"[score={h['score']:.3f}] {h['text']}" for h in hits])
    dt = round(time.perf_counter() - t0, 3)
    log_event("qa", {"query": req.query, "top_k": req.top_k, "latency_s": dt})
    return {"query": req.query, "latency_s": dt, "hits": hits, "answer": answer}

@app.post("/multihop")
def multihop(req: MultiHopRequest):
    """
    Simple 2-hop:
      1) First-hop retrieval on original query.
      2) Build a follow-up based on best hit title/id (if any) and do second retrieval.
      3) Merge & return.
    """
    if index.index is None:
        raise HTTPException(400, "Index not loaded. Upload corpus via app or set CORPUS_CSV.")
    t0 = time.perf_counter()
    first = index.search(req.query, k=req.top_k)
    bridge = first[0]["id"] if first else ""
    followup_q = f"{req.query}. Focus on {bridge}" if bridge else f"{req.query} (second hop)"
    second = index.search(followup_q, k=req.top_k)

    merged = (first + second)[: req.top_k]
    merged = sorted(merged, key=lambda x: -x["score"])

    synopsis = "\n\n---\n\n".join([f"[score={h['score']:.3f}] {h['text']}" for h in merged])
    dt = round(time.perf_counter() - t0, 3)
    log_event("multihop", {"query": req.query, "latency_s": dt})
    return {"query": req.query, "hop1_query": req.query, "hop2_query": followup_q, "latency_s": dt, "hits": merged, "synopsis": synopsis}

@app.get("/logs")
def logs():
    if not LOGS_PATH.exists():
        return {"events": []}
    return json.loads(LOGS_PATH.read_text())

@app.post("/sd/generate")
def sd_generate(req: SDRequest):
    if not SD_API_URL:
        raise HTTPException(400, "Set SD_API_URL env to your Track-A /generate endpoint.")
    try:
        r = requests.post(SD_API_URL, json=req.dict(), timeout=120)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        raise HTTPException(500, f"SD proxy failed: {e}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[RAG] Loaded corpus from: ./corpus.csv


In [2]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 66.4 MB/s eta 0:00:00


In [6]:
# app/app.py
import os
import io
import json
import base64
from pathlib import Path

import streamlit as st
import pandas as pd
import requests
from PIL import Image

BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8000")
OUT_DIRS = ["/content/outputs_sd", "outputs_sd", "./diffusion/outputs_sd"]  # where Track-A images might be

st.set_page_config(page_title="Week7 – Track-B", page_icon="🧭", layout="wide")
st.title("Week 7 — Track-B: Streamlit App + Project Models")

# ------------- Sidebar -------------
st.sidebar.header("Backend")
backend = st.sidebar.text_input("Backend URL", value=BACKEND_URL, help="FastAPI base URL")
st.sidebar.write("Health check:")
try:
    ok = requests.get(f"{backend}/health", timeout=10).json()
    st.sidebar.success(ok)
except Exception as e:
    st.sidebar.error(f"Backend not reachable: {e}")

st.sidebar.markdown("---")
st.sidebar.caption("Tip: set env BACKEND_URL when deploying.")

# ------------- Tabs -------------
tab1, tab2, tab3, tab4 = st.tabs(["Ask (RAG)", "Multi-hop", "Visualize (Week-7)", "Logs"])

# ---- Tab: Ask (RAG) ----
with tab1:
    st.subheader("Ask your corpus (single-hop RAG)")
    colL, colR = st.columns([2,1])
    with colR:
        st.markdown("### Upload corpus.csv")
        up = st.file_uploader("CSV with a text/content column", type=["csv"])
        if up is not None:
            # Push uploaded CSV to backend by saving and restarting it is out of scope in Streamlit;
            # Instead, we save locally for future runs (or switch to a config button/endpoint in a real app).
            save_path = Path("corpus.csv")
            save_path.write_bytes(up.getvalue())
            st.success(f"Saved corpus.csv to {save_path.resolve()}. Restart backend with env CORPUS_CSV={save_path.resolve()} to load it.")
    with colL:
        q = st.text_input("Your question", value="What does the dataset say about agent collaboration?")
        topk = st.slider("Top-K", 1, 10, 5)
        if st.button("Ask", use_container_width=True):
            try:
                r = requests.post(f"{backend}/qa", json={"query": q, "top_k": topk}, timeout=120)
                r.raise_for_status()
                data = r.json()
                st.success(f"Latency: {data['latency_s']} s")
                st.write("### Evidence")
                for h in data["hits"]:
                    st.markdown(f"**score:** {h['score']:.3f} • **id:** `{h['id']}`")
                    st.write(h["text"])
                    st.markdown("---")
                st.write("### Synthesis")
                st.code(data["answer"])
            except Exception as e:
                st.error(e)

# ---- Tab: Multi-hop ----
with tab2:
    st.subheader("Two-hop question answering")
    q = st.text_input("Complex question", value="How do agent teams compare to solo agents across temperature settings?")
    topk = st.slider("Top-K (per hop)", 1, 10, 5, key="mh_topk")
    if st.button("Run multi-hop", use_container_width=True):
        try:
            r = requests.post(f"{backend}/multihop", json={"query": q, "top_k": topk}, timeout=120)
            r.raise_for_status()
            data = r.json()
            st.success(f"Latency: {data['latency_s']} s")
            st.caption(f"Hop-1: `{data['hop1_query']}`")
            st.caption(f"Hop-2: `{data['hop2_query']}`")
            st.write("### Merged evidence")
            for h in data["hits"]:
                st.markdown(f"**score:** {h['score']:.3f} • **id:** `{h['id']}`")
                st.write(h["text"])
                st.markdown("---")
            st.write("### Synopsis")
            st.code(data["synopsis"])
        except Exception as e:
            st.error(e)

# ---- Tab: Visualize (Week-7) ----
with tab3:
    st.subheader("Visualize Week-7 Stable Diffusion outputs")
    found = []
    for d in OUT_DIRS:
        p = Path(d)
        if p.exists():
            found += list(p.glob("*.png"))
    if not found:
        st.info("No images found. Point OUT_DIRS to your Track-A outputs (e.g., /content/outputs_sd).")
    else:
        cols = st.columns(3)
        for i, img_path in enumerate(sorted(found)):
            with cols[i % 3]:
                st.image(str(img_path), caption=img_path.name, use_column_width=True)

    st.markdown("### Generate via Track-A endpoint")
    st.caption("Set SD_API_URL on backend; this UI just invokes /sd/generate (proxy).")
    prompt = st.text_input("Prompt", value="flat vector infographic of an agent workflow")
    if st.button("Generate (backend SD proxy)"):
        try:
            r = requests.post(f"{backend}/sd/generate", json={"prompt": prompt}, timeout=300)
            r.raise_for_status()
            b64 = r.json().get("image_base64")
            if b64:
                img = Image.open(io.BytesIO(base64.b64decode(b64)))
                st.image(img, caption="Generated via Track-A backend", use_column_width=True)
        except Exception as e:
            st.error(e)

# ---- Tab: Logs ----
with tab4:
    st.subheader("Recent backend events")
    try:
        r = requests.get(f"{backend}/logs", timeout=10)
        r.raise_for_status()
        data = r.json()
        st.code(json.dumps(data, indent=2)[:4000])
    except Exception as e:
        st.error(e)


2025-10-14 02:05:35.108 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-14 02:05:35.109 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-14 02:05:35.239 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-14 02:05:35.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-14 02:05:35.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-14 02:05:35.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-14 02:05:35.244 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [12]:
# ✅ Install Track-B deps (no watchdog pin; lets Streamlit pull a compatible <5)
%%bash
set -e

cat > requirements.txt << 'REQS'
# app + backend
streamlit==1.37.1
fastapi==0.115.5
uvicorn==0.31.1
python-dotenv==1.0.1
requests==2.32.4
pandas==2.2.2
Pillow==10.4.0

# retrieval
sentence-transformers==2.7.0
faiss-cpu==1.8.0

# tunnel
pyngrok==7.2.3
REQS

python -m pip -q install --upgrade pip
python -m pip -q install -r requirements.txt


In [13]:
import streamlit, requests, fastapi, uvicorn, starlette
print("streamlit:", streamlit.__version__)
print("requests:", requests.__version__)
print("fastapi:", fastapi.__version__)
print("uvicorn:", uvicorn.__version__)
print("starlette:", starlette.__version__)


streamlit: 1.50.0
requests: 2.32.4
fastapi: 0.118.2
uvicorn: 0.31.1
starlette: 0.48.0


In [14]:
# backend/main.py
%%bash
set -e
mkdir -p backend reports
cat > backend/main.py << 'PY'
import os, time, json
from typing import Optional, Dict, Any
from pathlib import Path

import pandas as pd
import numpy as np
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
import faiss, requests

# ---------- Config ----------
CORPUS_CSV = os.environ.get("CORPUS_CSV")                 # e.g., "/content/corpus.csv"
TOPK = int(os.environ.get("TOPK", "5"))
EMB_MODEL = os.environ.get("EMB_MODEL", "sentence-transformers/all-MiniLM-L6-v2")
SD_API_URL = os.environ.get("SD_API_URL")                 # Track-A /generate endpoint (optional)
LOGS_PATH = Path("reports/metrics.json"); LOGS_PATH.parent.mkdir(parents=True, exist_ok=True)

# ---------- Schemas ----------
class QARequest(BaseModel):
    query: str
    top_k: int = TOPK

class MultiHopRequest(BaseModel):
    query: str
    top_k: int = TOPK

class SDRequest(BaseModel):
    prompt: str
    negative_prompt: Optional[str] = "nsfw, watermark, logo"
    steps: int = 20
    guidance: float = 7.0
    height: int = 512
    width: int = 512
    seed: Optional[int] = 123

# ---------- Simple RAG Index ----------
class RAGIndex:
    def __init__(self, emb_model: str):
        self.model = SentenceTransformer(emb_model)
        self.df = None
        self.index = None
        self.text_col = None
        self.id_col = None

    def load_csv(self, path: str):
        df = pd.read_csv(path)
        # choose text col
        for c in ["text","content","chunk","passage","body","abstract","desc"]:
            if c in df.columns:
                self.text_col = c; break
        if self.text_col is None:
            str_cols = [c for c in df.columns if df[c].dtype==object]
            if not str_cols: raise ValueError("No text-like column found.")
            self.text_col = max(str_cols, key=lambda c: df[c].astype(str).str.len().sum())
        # optional id
        for c in ["id","doc_id","source","filename","title"]:
            if c in df.columns:
                self.id_col = c; break
        df = df[df[self.text_col].notna()].reset_index(drop=True)
        self.df = df

        embs = self.model.encode(df[self.text_col].astype(str).tolist(), batch_size=64,
                                 show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True)
        dim = embs.shape[1]
        idx = faiss.IndexFlatIP(dim)
        idx.add(embs.astype(np.float32))
        self.index = idx

    def search(self, q: str, k: int):
        if self.index is None: raise RuntimeError("Index not built.")
        q_emb = self.model.encode([q], convert_to_numpy=True, normalize_embeddings=True)
        D, I = self.index.search(q_emb.astype(np.float32), k)
        out=[]
        for s, i in zip(D[0], I[0]):
            row = self.df.iloc[int(i)]
            out.append({
                "score": float(s),
                "text": str(row[self.text_col]),
                "id": (str(row[self.id_col]) if self.id_col else str(int(i)))
            })
        return out

def log_event(kind: str, payload: Dict[str, Any]):
    now = time.time()
    data = {"events": []}
    if LOGS_PATH.exists():
        try: data = json.loads(LOGS_PATH.read_text())
        except: pass
    data["events"].append({"ts": now, "type": kind, "payload": payload})
    LOGS_PATH.write_text(json.dumps(data, indent=2))

# ---------- App ----------
app = FastAPI(title="Week7 Track-B Backend")
index = RAGIndex(EMB_MODEL)

# Try preload corpus
for p in [CORPUS_CSV, "./corpus.csv", "/content/corpus.csv"]:
    if p and Path(p).exists():
        try:
            index.load_csv(p)
            print(f"[RAG] Loaded: {p}")
            break
        except Exception as e:
            print("Preload error:", e)

@app.get("/health")
def health():
    return {"ok": True, "has_index": index.index is not None}

@app.post("/qa")
def qa(req: QARequest):
    if index.index is None: raise HTTPException(400, "Index not loaded. Upload via app or set CORPUS_CSV.")
    t0 = time.perf_counter()
    hits = index.search(req.query, k=req.top_k)
    ans = "Top evidence:\n\n" + "\n\n---\n\n".join([f"[{h['score']:.3f}] {h['text']}" for h in hits])
    dt = round(time.perf_counter()-t0, 3)
    log_event("qa", {"query": req.query, "latency_s": dt, "top_k": req.top_k})
    return {"query": req.query, "latency_s": dt, "hits": hits, "answer": ans}

@app.post("/multihop")
def multihop(req: MultiHopRequest):
    if index.index is None: raise HTTPException(400, "Index not loaded. Upload via app or set CORPUS_CSV.")
    t0 = time.perf_counter()
    hop1 = index.search(req.query, k=req.top_k)
    bridge = hop1[0]["id"] if hop1 else ""
    hop2_q = f"{req.query}. Focus on {bridge}" if bridge else f"{req.query} (second hop)"
    hop2 = index.search(hop2_q, k=req.top_k)
    merged = sorted((hop1 + hop2)[:req.top_k], key=lambda x: -x["score"])
    syn = "\n\n---\n\n".join([f"[{h['score']:.3f}] {h['text']}" for h in merged])
    dt = round(time.perf_counter()-t0, 3)
    log_event("multihop", {"query": req.query, "latency_s": dt})
    return {"query": req.query, "hop1_query": req.query, "hop2_query": hop2_q, "latency_s": dt, "hits": merged, "synopsis": syn}

@app.get("/logs")
def logs():
    if not LOGS_PATH.exists(): return {"events": []}
    return json.loads(LOGS_PATH.read_text())

@app.post("/sd/generate")
def sd_generate(req: SDRequest):
    if not SD_API_URL:
        raise HTTPException(400, "Set SD_API_URL env to your Track-A /generate endpoint.")
    try:
        r = requests.post(SD_API_URL, json=req.dict(), timeout=120)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        raise HTTPException(500, f"SD proxy failed: {e}")
PY


In [15]:
# app/app.py
%%bash
set -e
mkdir -p app diffusion reports
cat > app/app.py << 'PY'
import os, io, base64, json
from pathlib import Path
import requests, streamlit as st
from PIL import Image

BACKEND_URL = os.environ.get("BACKEND_URL", "http://127.0.0.1:8001")
OUT_DIRS = ["/content/outputs_sd", "outputs_sd", "./diffusion/outputs_sd"]

st.set_page_config(page_title="Week7 – Track-B", page_icon="🧭", layout="wide")
st.title("Week 7 — Track-B: Project App (RAG + Multihop + Visualize + Logs)")

# Sidebar
st.sidebar.header("Backend")
backend = st.sidebar.text_input("FastAPI base URL", value=BACKEND_URL)
try:
    hc = requests.get(f"{backend}/health", timeout=8).json()
    st.sidebar.success(hc)
except Exception as e:
    st.sidebar.error(f"Health failed: {e}")

tab1, tab2, tab3, tab4 = st.tabs(["Ask (RAG)", "Multi-hop", "Visualize (Week-7)", "Logs"])

# ------------ Ask (RAG) ------------
with tab1:
    st.subheader("Ask your corpus")
    colL, colR = st.columns([2,1], gap="large")
    with colR:
        st.markdown("### Upload corpus.csv")
        up = st.file_uploader("CSV with a text-like column", type=["csv"])
        if up is not None:
            p = Path("corpus.csv"); p.write_bytes(up.getvalue())
            st.success(f"Saved corpus.csv → {p.resolve()}\nRestart backend with env CORPUS_CSV={p.resolve()} to load.")
    with colL:
        q = st.text_input("Your question", value="What evidence discusses multi-agent collaboration?")
        topk = st.slider("Top-K", 1, 10, 5)
        if st.button("Ask", use_container_width=True):
            try:
                r = requests.post(f"{backend}/qa", json={"query": q, "top_k": topk}, timeout=120)
                r.raise_for_status()
                data = r.json()
                st.success(f"Latency: {data['latency_s']} s")
                st.write("### Evidence")
                for h in data["hits"]:
                    st.markdown(f"**score** {h['score']:.3f} • **id** `{h['id']}`")
                    st.write(h["text"]); st.markdown("---")
                st.write("### Synthesis")
                st.code(data["answer"])
            except Exception as e:
                st.error(e)

# ------------ Multihop ------------
with tab2:
    st.subheader("Two-hop reasoning")
    q2 = st.text_input("Complex question", value="Compare team vs solo agent outcomes across temperatures.")
    topk2 = st.slider("Top-K per hop", 1, 10, 5, key="mh_topk")
    if st.button("Run multi-hop", use_container_width=True):
        try:
            r = requests.post(f"{backend}/multihop", json={"query": q2, "top_k": topk2}, timeout=120)
            r.raise_for_status()
            data = r.json()
            st.success(f"Latency: {data['latency_s']} s")
            st.caption(f"Hop-1: `{data['hop1_query']}`")
            st.caption(f"Hop-2: `{data['hop2_query']}`")
            st.write("### Merged evidence")
            for h in data["hits"]:
                st.markdown(f"**score** {h['score']:.3f} • **id** `{h['id']}`")
                st.write(h["text"]); st.markdown("---")
            st.write("### Synopsis")
            st.code(data["synopsis"])
        except Exception as e:
            st.error(e)

# ------------ Visualize Week-7 ------------
with tab3:
    st.subheader("Visualize Track-A outputs")
    imgs=[]
    for d in OUT_DIRS:
        p=Path(d)
        if p.exists(): imgs += list(p.glob("*.png"))
    if not imgs:
        st.info("No images found. Point OUT_DIRS to Track-A outputs (e.g., /content/outputs_sd).")
    else:
        cols = st.columns(3)
        for i, im in enumerate(sorted(imgs)):
            with cols[i%3]:
                st.image(str(im), caption=im.name, use_column_width=True)

    st.markdown("### Generate via Track-A (proxy)")
    prompt = st.text_input("Prompt", value="flat vector infographic about agent workflow")
    if st.button("Generate (backend SD proxy)"):
        try:
            r = requests.post(f"{backend}/sd/generate", json={"prompt": prompt}, timeout=180)
            r.raise_for_status()
            b64 = r.json().get("image_base64")
            if b64:
                img = Image.open(io.BytesIO(base64.b64decode(b64)))
                st.image(img, caption="Generated image", use_column_width=True)
        except Exception as e:
            st.error(e)

# ------------ Logs ------------
with tab4:
    st.subheader("Recent events")
    try:
        r = requests.get(f"{backend}/logs", timeout=10)
        r.raise_for_status()
        st.code(json.dumps(r.json(), indent=2)[:4000])
    except Exception as e:
        st.error(e)
PY


In [16]:
# Run backend (FastAPI) + Streamlit, expose Streamlit with ngrok
import os, time, subprocess, signal
from pyngrok import ngrok

# Optional: point backend at your Track-A /generate
# os.environ["SD_API_URL"] = "http://127.0.0.1:8000/generate"

# Use your corpus automatically if present:
os.environ["CORPUS_CSV"] = os.environ.get("CORPUS_CSV", "/content/corpus.csv")

# Clean ports
for port in (8001, 8501):
    try:
        pid_txt = subprocess.check_output(["bash","-lc", f"lsof -t -i:{port} || true"]).decode().strip()
        if pid_txt:
            os.kill(int(pid_txt), signal.SIGKILL)
    except Exception:
        pass

# Backend
backend = subprocess.Popen(
    ["python","-m","uvicorn","backend.main:app","--host","0.0.0.0","--port","8001"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(3)

# Streamlit
env = os.environ.copy()
env["STREAMLIT_SERVER_HEADLESS"] = "true"
env["STREAMLIT_BROWSER_GATHER_USAGE_STATS"] = "false"
app = subprocess.Popen(
    ["streamlit","run","app/app.py","--server.address","0.0.0.0","--server.port","8501"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env
)
time.sleep(5)

# ngrok tunnel (set NGROK_AUTHTOKEN if you have one)
token = os.environ.get("NGROK_AUTHTOKEN", "33Ogb11oUbxV3VLlERb23EsAh3Y_6rwMqpHY4nrSuhAyr4Ndi")
if token:
    ngrok.set_auth_token(token)
tunnel = ngrok.connect(8501, "http")
print("Streamlit public URL:", tunnel.public_url)
print("Backend (local): http://127.0.0.1:8001")


Streamlit public URL: https://kelli-intercondylar-toi.ngrok-free.dev
Backend (local): http://127.0.0.1:8001


In [19]:
# Diagnose + (re)launch FastAPI backend at :8001 and wait until /health is ready

import os, time, signal, subprocess, textwrap, requests, sys, pathlib

# 0) Sanity: make sure the file exists
path = pathlib.Path("backend/main.py")
assert path.exists(), f"backend/main.py not found at {path.resolve()} — run the file creation cell again."

# 1) Kill any old listeners on 8001
try:
    pid_txt = subprocess.check_output(["bash","-lc", "lsof -t -i:8001 || true"]).decode().strip()
    if pid_txt:
        os.kill(int(pid_txt), signal.SIGKILL)
except Exception:
    pass

# 2) Start uvicorn in background but keep a handle to logs
env = os.environ.copy()
cmd = ["python","-m","uvicorn","backend.main:app","--host","0.0.0.0","--port","8001"]
print("Starting backend:", " ".join(cmd))
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)

# 3) Stream a few log lines while polling /health
backend_url = "http://127.0.0.1:8001/health"
ready = False
log_lines = []
t0 = time.time()
while time.time() - t0 < 60:  # up to 60s
    # read any available log lines without blocking too hard
    try:
        line = proc.stdout.readline()
        if line:
            log_lines.append(line.rstrip())
            if len(log_lines) <= 20:  # show first 20 immediately
                print(line.rstrip())
    except Exception:
        pass

    # if process exited, dump last logs and error out
    if proc.poll() is not None:
        rc = proc.returncode
        print("\n❌ Backend process exited (rc={}):".format(rc))
        tail = "\n".join(log_lines[-200:])
        print(tail)
        raise SystemExit(f"Backend crashed with return code {rc}. Check logs above.")

    # probe /health
    try:
        r = requests.get(backend_url, timeout=1.5)
        if r.ok:
            print("\n✅ /health:", r.json())
            ready = True
            break
    except Exception:
        pass

    time.sleep(1)

if not ready:
    # show more logs to help debug
    print("\n⚠️ Still not ready after 60s. Recent logs:")
    print("\n".join(log_lines[-200:]))
    raise SystemExit("Backend did not become healthy in time.")
else:
    print("Backend is up at:", backend_url.replace("/health",""))


Starting backend: python -m uvicorn backend.main:app --host 0.0.0.0 --port 8001
2025-10-14 02:25:16.499045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760408716.518947    6043 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760408716.524990    6043 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760408716.540038    6043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760408716.540059    6043 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.

✅ /hea

In [20]:
# Launch Streamlit properly and expose via ngrok, waiting for backend readiness first
import os, time, subprocess, signal, requests
from pyngrok import ngrok

BACKEND = "http://127.0.0.1:8001"
# Optional: point backend to Track-A /generate
# os.environ["SD_API_URL"] = "http://127.0.0.1:8000/generate"
os.environ["CORPUS_CSV"] = os.environ.get("CORPUS_CSV", "/content/corpus.csv")

# 1) Wait for backend health (quick poll, already started by previous cell)
for _ in range(30):
    try:
        if requests.get(BACKEND + "/health", timeout=1.5).ok:
            break
    except Exception:
        time.sleep(1)
else:
    raise SystemExit("Backend still not reachable at /health; run the fixer cell first.")

# 2) Kill any Streamlit on :8501
try:
    pid = subprocess.check_output(["bash","-lc","lsof -t -i:8501 || true"]).decode().strip()
    if pid: os.kill(int(pid), signal.SIGKILL)
except Exception:
    pass

# 3) Start Streamlit
env = os.environ.copy()
env["STREAMLIT_SERVER_HEADLESS"] = "true"
env["STREAMLIT_BROWSER_GATHER_USAGE_STATS"] = "false"
app = subprocess.Popen(
    ["streamlit","run","app/app.py","--server.address","0.0.0.0","--server.port","8501"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env
)

# 4) Open ngrok tunnel
token = os.environ.get("NGROK_AUTHTOKEN", "33Ogb11oUbxV3VLlERb23EsAh3Y_6rwMqpHY4nrSuhAyr4Ndi")
if token:
    ngrok.set_auth_token(token)
tunnel = ngrok.connect(8501, "http")
print("✅ Streamlit public URL:", tunnel.public_url)
print("Backend (local):", BACKEND)


✅ Streamlit public URL: https://kelli-intercondylar-toi.ngrok-free.dev
Backend (local): http://127.0.0.1:8001


In [18]:
# 🔧 Fix NumPy ABI mismatch for faiss/embeddings and relaunch-ready env
%%bash
set -e
python -m pip -q uninstall -y numpy faiss-cpu || true
python -m pip -q install "numpy==1.26.4"
python -m pip -q install "faiss-cpu==1.8.0"

# optional: re-pin sentence-transformers (pure-python; safe either way)
python -m pip -q install "sentence-transformers==2.7.0"


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
